In [64]:
import numpy as np
from scipy.integrate import cumulative_trapezoid
from scipy.signal import find_peaks
from scipy import signal
import plotly.express as px
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

In [65]:
import plotly.io as pio
pio.renderers.default = 'notebook_connected'

In [66]:
# Todo update this logic for files with name as prefix
data_dir = Path('./data/CPET hosp')
# data_dir = Path('./data/CPET ambu anonyme')
files = []
for xls in data_dir.glob('*.xlsx'):
    prefix = xls.name.split('_')[0]
    log_match = list(data_dir.glob(f'{prefix}*raw.log'))
    if len(log_match) == 0:
        print(f'No matching raw log for Excel file {xls}')
    elif len(log_match) > 1:
        print(f'More then 1 matching raw log for Excel file {xls}?')
    else:
        files.append((xls, log_match[0]))

In [67]:
xls, raw_log = files[5]

df_cycles = pd.read_excel(xls)
assert df_cycles['Unnamed: 0'][1] == 'Unité'
assert df_cycles['Unnamed: 0'][2] == 'Théor.'
# df_cycles = df_cycles.drop('Unnamed: 0', axis=1)
df_cycles = df_cycles.drop('Temps', axis=1)
df_cycles = df_cycles.drop([0, 1, 2])
keep_columns = {'Unnamed: 0', 'Vol.Cour.', 'VIn', 'tIn', 'VEx', 'tEx', 'ttot'}
df_cycles = df_cycles.drop([col for col in df_cycles.columns if col not in keep_columns], axis=1)
df_cycles = df_cycles.rename(columns={
    'Unnamed: 0': 'phase',
    'Vol.Cour.': 'vol_instant',
    'VIn': 'vol_in',
    'VEx': 'vol_ex',
    'tIn': 't_in',
    'tEx': 't_ex',
    'ttot': 'duration'
})
df_cycles['phase'] = df_cycles['phase'].replace('Repos', 'rest')
df_cycles['phase'] = df_cycles['phase'].replace('Charge', 'load')
df_cycles['phase'] = df_cycles['phase'].replace('Récupération', 'recovery')
df_cycles['phase'] = df_cycles['phase'].ffill()

# drop rows with missing/'-' values at the end
df_cycles = df_cycles.drop(df_cycles[df_cycles['vol_ex'] == '-'].index)
float_cols = ['vol_instant', 'vol_in', 't_in', 'vol_ex', 't_ex', 'duration']
for col in float_cols:
    df_cycles[col] = df_cycles[col].astype(float)
df_cycles = df_cycles.reset_index(drop=True)

In [68]:
xls, raw_log

(PosixPath('data/CPET hosp/240417_22_10_2020_9_38_ramp-test_excelrawdata.xlsx'),
 PosixPath('data/CPET hosp/240417_22_10_2020_9_38_ramp-test_raw.log'))

In [ ]:
df_cycles

In [ ]:
df_raw = pd.read_csv(raw_log, delimiter='\t', names=['t', 'flow', 'fo2', 'fco2'])
df_raw

In [71]:
sampling_freq = 1 / (df_raw['t'][:-1] - df_raw['t'].shift(1)[1:]).mean()
demean_window_seconds = 20

In [72]:
demean_window_seconds * sampling_freq

2500.0

In [73]:
filter_freq = 2*1e-6
1/(filter_freq * sampling_freq)

4000.0

In [74]:
# df_raw['flow_ma'] = df_raw['flow'].rolling(int(demean_window_seconds * sampling_freq)).mean().fillna(0)
# instant_vol = cumulative_trapezoid(y=df_raw['flow'] - df_raw['flow_ma'], x=df_raw['t'], initial=0)
df_raw['instant_vol_raw'] = cumulative_trapezoid(y=df_raw['flow'], x=df_raw['t'], initial=0) 
sos = signal.butter(4, Wn=filter_freq*sampling_freq, btype='highpass', output='sos')
flow_filtered = signal.sosfilt(sos, df_raw['flow'])
df_raw['instant_vol'] = cumulative_trapezoid(y=flow_filtered, x=df_raw['t'], initial=0) 

In [75]:
df_raw.instant_vol.sum()

-64.86788364850098

In [120]:
peaks, peakprops  = signal.find_peaks(df_raw['instant_vol'], prominence=0.20)
valls, vallprops = signal.find_peaks(-df_raw['instant_vol'], prominence=0.20)
# index of first peak that comes after first valley
first_peak_idx = np.argmin(peaks > valls[0])
# index of last peak that comes before last valley
last_peak_idx = np.argwhere(valls[-1] > peaks).max()
first_peak_idx, last_peak_idx
peaks = peaks[first_peak_idx:last_peak_idx+1]

#   p   p   p   n peaks
#  / \_/ \_/ \
# v   v   v   v n+1 valleys

# That makes n complete cycles

assert len(peaks) == len(valls) - 1
iv = df_raw['instant_vol']
vins = iv[peaks].array - iv[valls[:-1]].array
vexs = iv[peaks].array - iv[valls[1:]].array

In [121]:
pad_cycle_data = len(vins) - len(df_cycles['vol_in'])
cyc_vol_in_padded = np.pad(df_cycles['vol_in'].array, (pad_cycle_data, 0))
corr = np.correlate(cyc_vol_in_padded, vins, 'full')
shift = corr.argmax() - (len(vins) - 1)
drop_highres_cycles_front = pad_cycle_data - shift
drop_highres_cycles_back = shift 
vins_trimmed = vins[drop_highres_cycles_front:len(vins) - drop_highres_cycles_back]

In [128]:
peaks_trimmed = peaks[drop_highres_cycles_front:len(peaks) - drop_highres_cycles_back]
valls_trimmed = valls[drop_highres_cycles_front:len(valls) - drop_highres_cycles_back]
peaks_trimmed.shape, vins.shape, vins_trimmed.shape, valls_trimmed.shape
df_cycles['highres_t_start'] = df_raw['t'][valls_trimmed[:-1]].values
df_cycles['highres_t_max'] = df_raw['t'][peaks_trimmed].values
df_cycles['highres_t_end'] = df_raw['t'][valls_trimmed[1:]].values

In [ ]:
px.line(df_cycles, y=['duration', df_cycles['highres_t_end'] - df_cycles['highres_t_start']])

In [ ]:
df_raw.set_index('t')

In [ ]:
df_raw['vol_cycle_start'] = pd.Series(dtype=float)
df_raw.loc[df_cycles['highres_t_start'].values, 'vol_cycle_start'] = df_raw.loc[df_cycles['highres_t_start'].values]['instant_vol']
fig = px.line(df_raw, y='instant_vol')
fig = fig.add_scatter(x=df_raw.index, y=df_raw['vol_cycle_start'], mode='lines+markers')
fig.show()

In [ ]:
df_correlated = pd.DataFrame()
df_correlated['vol_in_highres'] = vins_trimmed
df_correlated['vol_in_cycles'] = df_cycles['vol_in']
px.scatter(df_correlated, df_correlated.index, ['vol_in_highres', 'vol_in_cycles'], title='Vol Insp cycles/highres')

In [ ]:
vexs_trimmed = vexs[drop_highres_cycles_front:-drop_highres_cycles_back]
cyc_vol_ex = df_cycles['vol_ex']
df_correlated['vol_ex_highres'] = vexs_trimmed
df_correlated['vol_ex_cycles'] = df_cycles['vol_ex']
px.scatter(df_correlated, df_correlated.index, ['vol_ex_highres', 'vol_ex_cycles'], title='Vol Exp cycles/highres')

In [ ]:
px.line(df_raw, x='t', y=['instant_vol_raw', 'instant_vol', 'flow'])
# px.scatter(df_raw, x='t', y=peaks)
# px.line(df_raw, x='t', y=['flow', 'flow_ma', 'filtered', 'instant_vol'])